# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [72]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [73]:
#Load the IMDB dataset. You can load it using pandas as dataframe
import csv
data=pd.read_csv("/content/IMDB Dataset.csv")

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [74]:
from bs4 import BeautifulSoup
def remove_html(text):
  soup= BeautifulSoup(text, "html.parser")
  return soup.get_text()
data['review']=data['review'].apply(remove_html)

In [75]:
def remove_url(text):
  return re.sub(r'http\S+','',text)
data['review']=data['review'].apply(remove_url)

In [76]:
def removeSpecialCharacters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text=text.lower()
    return text
data['review']=data['review'].apply(removeSpecialCharacters)

In [77]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def removeStopwordsandLemmatize(text):
  tokens=word_tokenize(text)
  tokens=[word for word in tokens if word not in stop_words]
  lemmatizer=WordNetLemmatizer()
  tokens=[lemmatizer.lemmatize(word) for word in tokens]
  tokens=' '.join(tokens)
  return tokens
data['review']=data['review'].apply(removeStopwordsandLemmatize)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [78]:
data.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive


In [112]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
avg=0
for i in range(len(data['review'])):
  avg+=(len(data['review'][i]))
avg=avg/len(data['review'])

In [113]:
print("Average length of sentences is ",avg)

Average length of sentences is  823.95158


In [117]:
data.describe()

,review,sentiment
count,50000,50000
unique,49578,2
top,loved today show variety solely cooking would ...,positive
freq,5,25000


# Naive Bayes classifier

In [80]:
# get reviews column from df
reviews = data['review']

# get labels column from df
labels = data['sentiment']

In [81]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

# print(enc.classes_)

In [133]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

train_sentences, test_sentences, train_labels, test_labels= train_test_split(reviews,encoded_labels,test_size=0.2,stratify=labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [103]:
for i in range(len(reviews)):
  train_corpus.append(reviews[i])

In [129]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

# max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_corpus)

In [128]:
X.shape

scipy.sparse.csr.csr_matrix

In [197]:
# Use laplace smoothing for words in test set not present in vocab of train set
# laplace smoothing is done while making the naive bayes model

In [189]:
from collections import Counter

In [ ]:
# Build the model. Don't use the model from sklearn
positive_text=""
reviews1= train_sentences.tolist()
label1= train_labels.tolist()
positive_text=""
for i in range(len(reviews1)):
  if(label1[i]==1):
    tokens=word_tokenize(reviews1[i])
    s=" ".join([word for word in tokens])
    positive_text+=s
  
negative_text=""
for i in range(len(reviews1)):
  if(label1[i]==0):
    tokens=word_tokenize(reviews1[i])
    s=" ".join([word for word in tokens])
    negative_text+=s

In [195]:
def word_count(text):
  words = re.split("\s+", text)
  return Counter(words)

positive_counts= word_count(positive_text)
negative_counts=word_count(negative_text)

positive_review_count= len([y for y in label1 if(y==1)])
negative_review_count= len([y for y in label1 if(y==0)])

prob_positive = positive_review_count / len(label1)
prob_negative = negative_review_count / len(label1)

In [196]:
def make_prediction(text, counts, class_prob, class_count):
  prediction = 1
  text_counts = Counter(re.split("\s+", text))
  for word in text_counts:
      prediction *=  text_counts.get(word) * ((counts.get(word, 0) + 1) / (sum(counts.values()) + class_count)) #LAPLACE SMOOTHING
  return prediction * class_prob

def final_prediction(text, make_prediction):
    negative_prediction = make_prediction(text, negative_counts, prob_negative, negative_review_count)
    positive_prediction = make_prediction(text, positive_counts, prob_positive, positive_review_count)
    if positive_prediction > negative_prediction:
      return 1
    return 0

In [198]:
# Test the model on test set and report Accuracy
reviews2=test_sentences.tolist()
predictions= [final_prediction(reviews2[i],make_prediction) for i in range(len(reviews2))]

In [199]:
print(predictions)
true_label= test_labels.tolist()
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(true_label, predictions, pos_label=1)
print("AUC of the predictions: {0}".format(metrics.auc(fpr, tpr)))

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 

# *LSTM* based Classifier

Use the above train and test splits.

In [200]:
# Hyperparameters of the model
vocab_size =10000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 800 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [201]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [202]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 800, 100)          1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_2 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 1,087,601
Trainable params: 1,087,601
Non-trainable params: 0
_________________________________________________________________


In [203]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 924s 821ms/step - loss: 0.3864 - accuracy: 0.8301 - val_loss: 0.3020 - val_accuracy: 0.8798
Epoch 2/5
1125/1125 [==============================] - 926s 823ms/step - loss: 0.2410 - accuracy: 0.9093 - val_loss: 0.2823 - val_accuracy: 0.8790
Epoch 3/5
1125/1125 [==============================] - 930s 827ms/step - loss: 0.1998 - accuracy: 0.9254 - val_loss: 0.3234 - val_accuracy: 0.8730
Epoch 4/5
1125/1125 [==============================] - 918s 816ms/step - loss: 0.1427 - accuracy: 0.9503 - val_loss: 0.3493 - val_accuracy: 0.8823
Epoch 5/5
1125/1125 [==============================] - 925s 822ms/step - loss: 0.1047 - accuracy: 0.9657 - val_loss: 0.3576 - val_accuracy: 0.8770


In [204]:
# Calculate accuracy on Test data

prediction = model.predict(test_padded)

# Get probabilities


import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
x= np.array(prediction)
softmax(x)
# Get labels based on probability 1 if p>= 0.5 else 0
l=[]
for p in x:
  if(p>=0.5):
    l.append(1)
  else:
    l.append(0)
# Accuracy : one can use classification_report from sklearn
target_names=['positive','negative']
print(classification_report(test_labels, l, target_names=target_names))

              precision    recall  f1-score   support

    positive       0.84      0.90      0.87      5000
    negative       0.89      0.82      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## Get predictions for random examples

In [ ]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
prediction=model.predict(padded)
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
x= np.array(prediction)
softmax(x)

# Get labels based on probability 1 if p>= 0.5 else 0

label=[]
for p in x:
  if(p>=0.5):
    label.append(1)
  else:
    label.append(0)
print(label)